In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
    run_parallel_transfer_to_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call

/home/cbisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**Choose the origin path where things need to be transferred from**

In [4]:
# directory = directory_project
# directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'
directory = "dropbox"
directory = "/projects/0/einf914/agg/"

# update_plate_info("directory")

In [5]:
all_folders = get_current_folders(directory)

**Select the folders of interest within the folders at the path location**

In [9]:
folders = all_folders.loc[all_folders["Plate"] == 737]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
8142,/projects/0/einf914/agg/20210903_2341_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,True,False,False,False,False,False,"03.09.2021, 23:41:",20210903_2341_Plate34,NaN
8143,/projects/0/einf914/agg/20210908_1302_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,True,False,False,False,False,False,"08.09.2021, 13:02:",20210908_1302_Plate34,NaN
8144,/projects/0/einf914/agg/20210911_1058_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,False,False,False,False,False,False,"11.09.2021, 10:58:",20210911_1058_Plate34,NaN
8145,/projects/0/einf914/agg/20210904_2341_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,True,False,False,False,False,False,"04.09.2021, 23:41:",20210904_2341_Plate34,NaN
8146,/projects/0/einf914/agg/20210907_1348_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,True,False,False,False,False,False,"07.09.2021, 13:48:",20210907_1348_Plate34,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,/projects/0/einf914/agg/20210920_1159_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,False,False,False,False,False,False,"20.09.2021, 11:59:",20210920_1159_Plate34,NaN
8358,/projects/0/einf914/agg/20210930_0349_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,True,False,False,False,False,False,"30.09.2021, 03:49:",20210930_0349_Plate34,NaN
8359,/projects/0/einf914/agg/20211011_2012_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,False,False,False,False,False,False,"11.10.2021, 20:12:",20211011_2012_Plate34,NaN
8360,/projects/0/einf914/agg/20210911_2254_Plate34,737,34,Carrot,Agg,100P100N,Y,-,20210901,True,...,True,False,False,False,False,False,False,"11.09.2021, 22:54:",20210911_2254_Plate34,NaN


**Choose the target path where things need to be transferred to**

In [36]:
directory_targ = os.path.join(directory_scratch, "temp") + "/"
# directory_targ = os.path.join(directory_archive,'Prince_Data')+'/'
directory_targ = "/projects/0/einf914/agg/"

**To transfer data from surfsara to dropbox**
- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go


In [10]:
dir_drop = "prince_data"
run_parallel_transfer(
    "toward_drop.py",
    [directory, dir_drop],
    folders,
    5,
    "20:00",
    "staging",
    cpus=1,
    node="staging",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 311635
Submitted batch job 311636
Submitted batch job 311637
Submitted batch job 311638
Submitted batch job 311639
Submitted batch job 311640


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 311641
Submitted batch job 311642
Submitted batch job 311643
Submitted batch job 311644
Submitted batch job 311645
Submitted batch job 311646


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a n

Submitted batch job 311665
Submitted batch job 311666
Submitted batch job 311667
Submitted batch job 311668
Submitted batch job 311669
Submitted batch job 311670
Submitted batch job 311671
Submitted batch job 311672
Submitted batch job 311673
Submitted batch job 311674
Submitted batch job 311675
Submitted batch job 311676


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 311677
Submitted batch job 311678
Submitted batch job 311679


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


**To transfer data from dropbox to surfsara**
- use the 'from_drop.py' function including the folders of interest
- unzip argument decides if folders must also be unzipped
- flatten argument decides if folders must be flatten to respect snellius paralell computing logic or kept in a folder for transfer to archive for example

In [37]:
dir_drop = "prince_data"
unzip = True
flatten = True
run_parallel_transfer(
    "from_drop.py",
    [directory_targ, dir_drop, unzip, flatten],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275403
Submitted batch job 275404
Submitted batch job 275405


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275406
Submitted batch job 275407
Submitted batch job 275408
Submitted batch job 275409
Submitted batch job 275411
Submitted batch job 275412


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275443
Submitted batch job 275444
Submitted batch job 275445


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 275446
Submitted batch job 275447
Submitted batch job 275448
Submitted batch job 275449
Submitted batch job 275450
Submitted batch job 275451


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [6]:
unzip = "False"
flatten = False
dir_drop = "prince_data"

directory_targ = os.path.join(directory_scratch, "temp") + "/"
directory = directory_targ
run_parallel_transfer(
    "from_drop.py",
    [directory_targ, dir_drop, unzip, flatten],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="archiving.sh",
)
run_parallel_transfer_to_archive(
    folders, directory, "10:00", "staging", name_job="archiving.sh"
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298787
Submitted batch job 298788
Submitted batch job 298789
Submitted batch job 298790
Submitted batch job 298791
Submitted batch job 298792


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298793
Submitted batch job 298794
Submitted batch job 298795
Submitted batch job 298796
Submitted batch job 298797


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298826
Submitted batch job 298827
Submitted batch job 298828
Submitted batch job 298829
Submitted batch job 298830


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 298831
Submitted batch job 298832
Submitted batch job 298833
Submitted batch job 298834
Submitted batch job 298835


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
